In [ ]:
pip install pandasql

In [ ]:
pip install robin_stocks

In [4]:
import robin_stocks.robinhood as r
from robin_stocks.globals import LOGGED_IN, SESSION, OUTPUT
import pyotp
import requests
import json
import pandas as pd
import pandasql as ps
from pandasql import sqldf
from datetime import datetime
import yfinance as yf
from google.colab import drive

In [15]:
# Function to log-in user
def login_function(username, password, mfa_code):
  try:
    r.authentication.login(username,password,mfa_code=mfa_code,expiresIn=900)
  except Exception as e:
    print("Error: ", e)

In [6]:
# Function to get raw dataset with all filled orders

"""Takes a stock instrument url and returns the ticker symbol associated with the stock.
    :param id: A string that represents the stocks instrument url.
    :type id: str
    :returns:  The symbol.
    """
symbol_for_stock = lambda url: requests.get(url).json()['symbol']

In [23]:
# Function to get raw dataset with all filled orders
def get_all_orders():

  filled_orders = []
  filled_orders.append(['symbol','latest_price', 'order_type','quantity','unit_price','total_price','fees','timestamp'])

  stocks_etf = r.orders.get_all_stock_orders()
  stock_prices = {}
  for order in stocks_etf:
    if order['executions'] != []:
      order_details = []
      order_details.append(symbol_for_stock(order['instrument']))
      try:
        price = stock_prices[symbol_for_stock(order['instrument'])]
      except:
        try:
          price = r.stocks.get_latest_price(symbol_for_stock(order['instrument']), includeExtendedHours=True)[0]
        except:
          price = 0
        stock_prices[symbol_for_stock(order['instrument'])] = price
      order_details.append(price)
      order_details.append(order['side'])
      order_details.append(order['cumulative_quantity'])
      order_details.append(order['average_price'])
      order_details.append(order['executed_notional']['amount'])
      order_details.append(order['fees'])
      order_details.append(order['updated_at'])
      filled_orders.append(order_details)

  crypto = r.orders.get_all_crypto_orders()
  crypto_prices = {}
  for order in crypto:
    if order['executions'] != []:
      order_details = []
      order_details.append(order['currency_code'])
      try:
        price = crypto_prices[order['currency_code']]
      except:
        price = r.crypto.get_crypto_quote(order['currency_code'])['mark_price']
        crypto_prices[order['currency_code']] = price
      order_details.append(price)
      order_details.append(order['side'])
      order_details.append(order['cumulative_quantity'])
      order_details.append(order['average_price'])
      order_details.append(order['rounded_executed_notional'])
      order_details.append(0)
      order_details.append(order['updated_at'])
      filled_orders.append(order_details)

  # Converting to dataframe for analysis
  order_data = pd.DataFrame(filled_orders[1:], columns=filled_orders[0])

  # Ensure data types are correct
  order_data['timestamp'] = pd.to_datetime(order_data['timestamp'],utc=True)
  order_data = order_data.sort_values(by='timestamp', ascending=True)
  order_data[['quantity', 'unit_price', 'total_price', 'fees', 'latest_price']] = order_data[['quantity', 'unit_price', 'total_price', 'fees', 'latest_price']].astype(float)

  return order_data

In [8]:
# Function to adjust for stock splits
def adjust_for_stock_splits(data):
  adjusted_data = data.copy()
  symbols = adjusted_data['symbol'].unique()

  for symbol in symbols:
      stock = yf.Ticker(symbol)
      splits = stock.splits

      if splits.empty:
          continue

      for split_date, split_ratio in splits.items():
          # Transactions before the split date are adjusted
          transactions_to_adjust = adjusted_data[(adjusted_data['symbol'] == symbol) & (adjusted_data['timestamp'] < split_date)]
          transactions_after = adjusted_data[(adjusted_data['symbol'] == symbol) & (adjusted_data['timestamp'] >= split_date)]

          adjusted_data.loc[transactions_to_adjust.index, 'quantity'] *= split_ratio
          adjusted_data.loc[transactions_to_adjust.index, 'unit_price'] /= split_ratio
          # adjusted_data.loc[transactions_to_adjust.index, 'total_price'] = adjusted_data.loc[transactions_to_adjust.index, 'quantity'] * adjusted_data.loc[transactions_to_adjust.index, 'unit_price']

  return adjusted_data

In [9]:
# Function to filter transactions by user-defined period
def filter_transactions_by_period(data, start_date, end_date):
  # Convert strings to datetime
  start_date = pd.to_datetime(start_date,utc=True)
  end_date = pd.to_datetime(end_date,utc=True)

  # Filter data
  filtered_data = data[(data['timestamp'] >= start_date) & (data['timestamp'] <= end_date)]
  return filtered_data

In [33]:
# Main analysis function
def perform_investment_analysis(data, start_date=None, end_date=None):

  # Initialize the summary table
  summary_table = pd.DataFrame(columns=['symbol', 'quantity_purchased', 'investment_amount', 'quantity_sold', 'return_amount', 'realized_gain_loss', 'unrealized_gain_loss'])


  # Adjust data for stock splits
  data = adjust_for_stock_splits(data)

  # Filter data for the user-defined period
  if not start_date: start_date = '2013-04-18'
  if not end_date: end_date = datetime.datetime.now()
  data = filter_transactions_by_period(data, start_date, end_date)


  # Re-process each symbol with corrected logic
  for symbol in data['symbol'].unique():

      # Track realized gains/losses separately for each year
      realized_gains_losses = {}

      symbol_data = data[data['symbol'] == symbol]
      buy_orders = symbol_data[symbol_data['order_type'] == 'buy'].copy()
      sell_orders = symbol_data[symbol_data['order_type'] == 'sell'].copy()

      # Reset trackers for summary data
      total_quantity_purchased = buy_orders['quantity'].sum()
      total_investment_amount = buy_orders['total_price'].sum() - buy_orders['fees'].sum()
      total_quantity_sold = sell_orders['quantity'].sum()
      total_return_amount = sell_orders['total_price'].sum() - sell_orders['fees'].sum()
      realized_gain_loss = 0


      # Adjust FIFO calculation for realized gains/losses to handle no corresponding buy orders
      for index, sell_order in sell_orders.iterrows():
          fiscal_year = sell_order['timestamp'].year
          sell_quantity = sell_order['quantity']

          while sell_quantity > 0 and not buy_orders[buy_orders['quantity'] > 0].empty:
              buy_order = buy_orders[buy_orders['quantity'] > 0].iloc[0]
              buy_quantity = buy_order['quantity']

              quantity_to_sell = min(sell_quantity, buy_quantity)
              realized_gain_loss = (sell_order['unit_price'] - buy_order['unit_price']) * quantity_to_sell

              # Update realized gains/losses by fiscal year
              if fiscal_year not in realized_gains_losses:
                  realized_gains_losses[fiscal_year] = realized_gain_loss
              else:
                  realized_gains_losses[fiscal_year] += realized_gain_loss

              sell_quantity -= quantity_to_sell
              buy_orders.at[buy_order.name, 'quantity'] -= quantity_to_sell

      # Adjust calculation for unrealized gains/losses
      unrealized_gain_loss = 0
      for index, buy_order in buy_orders.iterrows():
          if buy_order['quantity'] > 0:  # Only unsold shares are considered
              current_price_per_unit = symbol_data.iloc[-1]['latest_price']
              buy_price_per_unit = buy_order['unit_price']
              unrealized_gain_loss += (current_price_per_unit - buy_price_per_unit) * buy_order['quantity']
              # print("Unrealized gain/loss: ", unrealized_gain_loss)

      # Append corrected data to the summary table
      summary_table = pd.concat([summary_table, pd.DataFrame({
          'symbol': [symbol],
          'quantity_purchased': [total_quantity_purchased],
          'investment_amount': [total_investment_amount],
          'quantity_sold': [total_quantity_sold],
          'return_amount': [total_return_amount],
          'realized_gain_loss': [realized_gains_losses],
          'unrealized_gain_loss': [unrealized_gain_loss]
      })], ignore_index=True)

  return summary_table


In [69]:
# Function to pull annual realized gains and losses

def annual_realized_summary(df):

  summary_table = df.copy()

  # Extracting and normalizing the realized_gain_loss column
  realized_gains = pd.json_normalize(summary_table['realized_gain_loss'])

  # Merging the normalized realized gains with the original symbol column
  realized_gains['symbol'] = summary_table['symbol']

  # Melting the DataFrame to have one row per symbol-year pair
  melted_gains = realized_gains.melt(id_vars=['symbol'], var_name='year', value_name='gain_loss').dropna()
  
  # Summarizing by year
  yearly_summary = melted_gains.groupby('year')['gain_loss'].agg(['sum', 'idxmax', 'idxmin'])
  yearly_summary.columns = ['Total Realized Gains', 'Top Gainer', 'Top Loser']

  # Adding symbol information to yearly summary
  yearly_summary['Top Gainer'] = melted_gains.loc[yearly_summary['Top Gainer'], 'symbol'].values
  yearly_summary['Top Loser'] = melted_gains.loc[yearly_summary['Top Loser'], 'symbol'].values

  # Group by year and apply a function to get top 5 gainers or losers
  grouped = melted_gains.groupby('year', as_index=False)

  # For gainers, sort in descending order
  yearly_top_5 = grouped.apply(lambda x: x[x['gain_loss'] > 0].nlargest(5, 'gain_loss')).reset_index(drop=True)

  # For losers, sort in ascending order
  yearly_bottom_5 = grouped.apply(lambda x: x.nsmallest(5, 'gain_loss')).reset_index(drop=True)

  # All-time metrics
  all_time_total_invested = summary_table['investment_amount'].sum()
  all_time_total_capital_returned = summary_table['return_amount'].sum()
  all_time_unrealized_gains_losses = summary_table['unrealized_gain_loss'].sum()
  all_time_realized_gains_losses = yearly_summary['Total Realized Gains'].sum()

  return yearly_summary, yearly_top_5, yearly_bottom_5, all_time_total_invested, all_time_total_capital_returned, all_time_unrealized_gains_losses, all_time_realized_gains_losses


In [ ]:
# Function compiling all previous functions into one
def final_results(username, password, filepath, mfa_code=None, start_date=None, end_date=None):
  try:
    login = login_function(username,password,mfa_code=mfa_code)
    data = get_all_orders()
    summary = perform_investment_analysis(data, start_date, end_date)
    summary_annual, top_5, bottom_5, all_time_total_invested, all_time_total_capital_returned, all_time_unrealized_gains_losses, all_time_realized_gains_losses = annual_realized_summary(summary)

    print("All-time Total Money Invested:", all_time_total_invested)
    print("All-time Total Money Returned:", all_time_total_capital_returned)
    print("All-time Unrealized Gains/Losses:", all_time_unrealized_gains_losses)
    print("All-time Realized Gains/Losses:", all_time_realized_gains_losses)

    print("\nReturns per year: ")
    display(summary_annual)
    print("\nTop 5 gainers per year: ")
    display(top_5)
    print("\nBottom 5 losers per year: ")
    display(bottom_5)

    summary.to_csv(filepath, index=False)
  except Exception as e:
      return ("Error: ", e)

In [ ]:
# Print Annual Summary and export all-time summary table to csv
username = "Enter your username"
password = "Enter your password"
mfa_code = "Enter you mfa code"
filepath = "Enter your filepath" #e.g., - '/content/drive/MyDrive/Colab Notebooks/RH_investment_summary.csv'
start_date = "Enter your start date" #e.g., - '2020-01-01' (default = '2013-04-18)
end_date = "Enter your end date" #e.g., - '2023-12-31' (default = datetime.now())
final_results(username, password, filepath, mfa_code=mfa_code, start_date=start_date, end_date=end_date) # If you don't have mfa_code and don't want to specify start_date or end_date, then just omit them from the function call